In [1]:
import pandas as pd
import os
import random
import math
import json

In [12]:
candidatos = os.listdir('/home/alysson/Downloads/Tweets')
candidatos.remove('.ipynb_checkpoints')
candidatos.remove('sampling.ipynb')
candidatos.remove('bots.ipynb')
candidatos.remove('qtd.py')
candidatos.remove('MAIA')
candidatos.remove('detection_bots.py')
candidatos.remove('SCREEN_NAME.csv')
candidatos.remove('SCREEN_NAME_BOT.csv')
candidatos.remove('ELEICOES')
candidatos.remove('.idea')
candidatos

['BOLSONARO', 'ALCKMIN', 'CIRO', 'TEMER', 'MANUELA', 'LULA', 'MARINA']

In [23]:
columns = ['id','tweet','candidato']

for c in candidatos:
    files = os.listdir('/home/alysson/Downloads/Tweets/'+c)
    size = len(files)
    print(size)
    indexes = random.sample(range(0,size), 200)
    samples = []
    

    for i in indexes:
        with open(c+'/'+files[i]) as data_file:
            data_string = data_file.read()    
            try:
                values = []
                
                tweet = json.loads(data_string)
                text_aux = ""
                
                if tweet['is_quote_status'] == 'true':
                    if 'extended_tweet' in tweet['quoted_status']:
                        text_aux = tweet['text'] + tweet['quoted_status']['extended_tweet']['full_text']
                    else:
                        text_aux = tweet['text'] + tweet['quoted_status']['text']
                elif 'retweeted_status' in tweet:
                    if 'extended_tweet' in tweet['retweeted_status']:
                        text_aux = tweet['retweeted_status']['extended_tweet']['full_text']
                    else:
                        text_aux = tweet['retweeted_status']['text']
                else:
                    text_aux = tweet['text']
                
                values.append(str('twe'+tweet['id_str']))
                values.append(text_aux)
                values.append(c.lower())
                samples.append(values)
            except ValueError:
                print('Failed:')
                print(repr(data_string))
    
    df = pd.DataFrame(data=samples, columns=columns)
    df.to_csv(c+'.csv',sep='\t', index=False)

    print(c+' foi salvo com 200 tweets')

16383
BOLSONARO foi salvo com 200 tweets
162552
ALCKMIN foi salvo com 200 tweets
3380
CIRO foi salvo com 200 tweets
16383
TEMER foi salvo com 200 tweets
2790
MANUELA foi salvo com 200 tweets
5861
LULA foi salvo com 200 tweets
18581
MARINA foi salvo com 200 tweets


In [14]:
with open('ELEICOES/tweets_eleicoes.json') as data_file:
    data_string = data_file.read()
    tweets = json.loads(data_string)
    print(len(tweets))

5232


In [24]:
samples = []

indexes = random.sample(range(0,len(tweets)), 200)

for index in indexes:
    tweet = tweets[index]
    try:
        values = []
        text_aux = ""

        if tweet['is_quote_status'] == 'true':
            if 'extended_tweet' in tweet['quoted_status']:
                text_aux = tweet['text'] + tweet['quoted_status']['extended_tweet']['full_text']
            else:
                text_aux = tweet['text'] + tweet['quoted_status']['text']
        elif 'retweeted_status' in tweet:
            if 'extended_tweet' in tweet['retweeted_status']:
                text_aux = tweet['retweeted_status']['extended_tweet']['full_text']
            else:
                text_aux = tweet['retweeted_status']['text']
        else:
            text_aux = tweet['text']

        values.append(str('twe'+tweet['id_str']))
        values.append(text_aux)
        values.append('eleicoes')
        samples.append(values)
    except ValueError:
        print('Failed:')
        print(repr(data_string))
    
df = pd.DataFrame(data=samples, columns=columns)
df.to_csv('ELEICOES.csv',sep='\t',encoding='utf-8', index=False)

## Amostra para avaliação de auto-consistência e consistência entre os marcadores
-----------
#### Processo
* Pegar todos arquivos e juntar em um data frame;
* Aleatorizar as linhas e dividir o data frame em 4 partes;
* Para cada data frame $D_i$, pegue 10 linhas aleatórios e dupliques no data frame $D_i$;
* Para cada data frame $D_i$, pegar as 20 primeiras linhas dos data frames $D_{i+1 \mod 4}$, $D_{i+2 \mod 4}$ e $D_{i+3 \mod 4}$.

In [25]:
smpls = os.listdir('/home/alysson/Downloads/Tweets/SAMPLES/')
smpls

['MANUELA.csv',
 'LULA.csv',
 'CIRO.csv',
 'TEMER.csv',
 'MARINA.csv',
 'BOLSONARO.csv',
 'ALCKMIN.csv']

In [43]:
dfs = []
for s in smpls:
    dfs.append(pd.read_csv('/home/alysson/Downloads/Tweets/SAMPLES/'+s, sep='\t'))

In [44]:
df = pd.concat(dfs) #concatenar data frames

In [58]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle entre as linhas

In [46]:
marcadores = ['alysson','dalai','romulo','raul']

In [47]:
df['marcador'] = ''

In [64]:
c = 0
for tweet in df.values:
    tweet[3] = marcadores[c%4]
    c += 1

In [69]:
alysson = df[df['marcador'] == 'alysson']
dalai = df[df['marcador'] == 'dalai']
romulo = df[df['marcador'] == 'romulo']
raul = df[df['marcador'] == 'raul']

In [70]:
dtframes = [alysson, dalai, romulo, raul]

In [77]:
i = 0
for d in dtframes:
    dtframes[i] = pd.concat([d,d[0:10]])
    i += 1

In [80]:
dtframes[0][0:20]

,id,tweet,candidato,marcador
0,twe977318071722442752,"@jangodurango ""Se eu soubesse que tu queria nã...",lula,alysson
4,twe978104151359008769,Retweeted Florisvaldo Raimundo (@Florisrs):\n\...,lula,alysson
8,twe984802485360582656,Ao invés dos petistas estarem buscando livrar ...,alckmin,alysson
12,twe977572021361168389,@CshmKnCaioHen @Peaotrabalhador @MiguelAMSA61 ...,manuela,alysson
16,twe977558211447443457,"A pergunta é séria, @manudeputada: quer me pag...",manuela,alysson
20,twe977347407011897345,DISPARADO!!! 90%... https://t.co/f5G9063duC,bolsonaro,alysson
24,twe984494027956543488,O que acontece agora que o inquérito de Alckmi...,alckmin,alysson
28,twe981677295676084224,"- ""aécio é flagrado pedindo grana a empresário...",temer,alysson
32,twe978200570740822017,Se a bunda de algum ministro sentar sobre o pr...,lula,alysson
36,twe982682347484180482,"Hoje, não há lado certo ou lado errado. Lula é...",temer,alysson


In [81]:
i = 0
for d in dtframes:
    dtframes[i] = pd.concat([dtframes[i], dtframes[(i+1)%4][0:20]])
    dtframes[i] = pd.concat([dtframes[i], dtframes[(i+2)%4][0:20]])
    dtframes[i] = pd.concat([dtframes[i], dtframes[(i+3)%4][0:20]])
    i+=1

In [86]:
for i in range(len(dtframes)):
    print(dtframes[i].groupby('marcador').count())
    print('----------------------------------------')

           id  tweet  candidato
marcador                       
alysson   360    360        360
dalai      20     20         20
raul       20     20         20
romulo     20     20         20
----------------------------------------
           id  tweet  candidato
marcador                       
alysson    20     20         20
dalai     360    360        360
raul       20     20         20
romulo     20     20         20
----------------------------------------
           id  tweet  candidato
marcador                       
alysson    20     20         20
dalai      20     20         20
raul       20     20         20
romulo    360    360        360
----------------------------------------
           id  tweet  candidato
marcador                       
alysson    20     20         20
dalai      20     20         20
raul      360    360        360
romulo     20     20         20
----------------------------------------


In [90]:
i = 0
for d in dtframes:
    d.to_csv(marcadores[i]+".csv", sep=';', index=False)
    i += 1